# Introduction to Foundation Models

![foundation](https://cdn.midjourney.com/12b38a83-119d-4c5c-949d-2fa3ebfe0140/0_2.png)

## Table of Contents

1. Overview
2. Applications
3. Training a Model
    - NumPy Example
    - PyTorch Example
5. Transfer Learning
    - NumPy Example
    - PyTorch Example
6. Transformers
    - Tokenization
    - Training Process
    - Prompting
    - Fine-Tuning
    - RLHF
    - Challenges
7. Serving
8. What we are up to at Seldon
9. Conclusion

## 1. Overview

Foundation models are large-scale machine learning models trained on vast amounts of data, 
enabling them to adapt to various downstream tasks with minimal fine-tuning. These models 
have turned the field of AI upside down by providing a powerful starting point for a wide 
range of applications powered with automation intelligent components in them. Some 
key characteristics of foundation models include:

1. Pre-training on massive datasets
2. Ability to transfer knowledge to new tasks
3. Capability to handle multiple modalities (e.g., text, images, audio, tabular)
4. Scalability and efficiency in training and deployment

## 2. Applications

Foundation models have found applications across various domains, including:

1. Natural Language Processing (NLP): Language translation, text summarization, sentiment analysis, question answering, and more.
2. Computer Vision: Image classification, object detection, semantic segmentation, and image generation.
3. Speech Recognition: Automatic speech recognition, speaker identification, and voice synthesis.
4. Multimodal Learning: Combining multiple modalities, such as text and images, for tasks like image captioning and visual question answering.

## 3. Training a Model

### 3.1 NumPy Model

In the example below, we define a `TinyImageClassifier` class that encapsulates the model parameters, 
forward pass, backward pass, training loop, and prediction method.

1. The `__init__` method initializes the model parameters (W1, b1, W2, b2) based on the input size, hidden size, and number of classes. Think of the neural network as a team of workers in a factory. Each worker (neuron) has a specific task and is connected to other workers. The weights represent the strength of the connections between workers, and the biases represent the individual preferences of each worker. Initially, the workers are assigned random tasks (weights) and have no specific preferences (biases). As the training progresses, the workers learn and adapt their tasks and preferences based on the feedback they receive.
2. The forward method performs the forward pass, computing the output scores given the input data X. Imagine the neural network as a series of conveyor belts in a factory. The input data (X) is placed on the first conveyor belt, and as it moves along, it undergoes transformations. The weights (W1 and W2) represent the machinery that processes the data, and the biases (b1 and b2) are additional adjustments made to the data. The ReLU activation function is like a quality control checkpoint that filters out any negative values. Finally, the processed data reaches the end of the conveyor belt, resulting in the output predictions.
3. The backward method computes the gradients of the loss with respect to the weights and biases using 
[the chain rule](https://machinelearningmastery.com/the-chain-rule-of-calculus-for-univariate-and-multivariate-functions/). Think of the loss as a measure of how far off the mark the model's predictions are from the true targets. Imagine you are an archer aiming at a target. The output scores are like the arrows you shoot, and the true labels are the bullseye. The softmax function normalizes the scores, similar to adjusting the tension in your bow to ensure the arrows land on the target. The cross-entropy loss measures the distance between your arrows and the bullseye. The smaller the loss, the closer your arrows are to the center of the target.
4. The train method implements the training loop, where it iteratively performs the forward pass, computes the loss, 
performs the backward pass, and updates the weights and biases using gradient descent. Think of the training loop as a fitness program. Each epoch is like a training session where you exercise (perform forward and backward passes) to improve your fitness level (reduce the loss). The weights and biases are like your muscles, and the gradient descent is the training regimen that strengthens them. After each session, you assess your progress (loss) to see how much you've improved. You repeat this process for a set number of sessions (epochs) until you reach your desired fitness level (minimum loss).
5. The predict method allows you to make predictions on new samples. It takes the input data X, performs the forward 
pass, applies the softmax function to the output scores, and returns the predicted class labels.

In [ ]:
import numpy as np

class TinyImageClassifier:
    def __init__(self, input_size, hidden_size, num_classes):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, num_classes) * 0.01
        self.b2 = np.zeros((1, num_classes))
    
    def forward(self, X):
        self.h = np.maximum(0, np.dot(X, self.W1) + self.b1)
        scores = np.dot(self.h, self.W2) + self.b2
        return scores
    
    def backward(self, X, y, scores):
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        num_samples = X.shape[0]
        
        dscores = probs
        dscores[range(num_samples), y] -= 1
        dscores /= num_samples
        
        dW2 = np.dot(self.h.T, dscores)
        db2 = np.sum(dscores, axis=0, keepdims=True)
        
        dh = np.dot(dscores, self.W2.T)
        dh[self.h <= 0] = 0
        
        dW1 = np.dot(X.T, dh)
        db1 = np.sum(dh, axis=0, keepdims=True)
        
        return dW1, db1, dW2, db2
    
    def train(self, X, y, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            scores = self.forward(X)
            
            exp_scores = np.exp(scores)
            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
            correct_logprobs = -np.log(probs[range(num_samples), y])
            loss = np.sum(correct_logprobs) / num_samples
            
            dW1, db1, dW2, db2 = self.backward(X, y, scores)
            
            self.W1 -= learning_rate * dW1
            self.b1 -= learning_rate * db1
            self.W2 -= learning_rate * dW2
            self.b2 -= learning_rate * db2
            
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")
    
    def predict(self, X):
        scores = self.forward(X)
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return np.argmax(probs, axis=1)

To use this model, you can create an instance of the TinyImageClassifier class, specifying the input size, hidden size, 
and number of classes. Then, you can call the train method to train the model on your training data X and labels y.

After training, we can use the predict method to make predictions on new samples X_test. The predicted class labels will be returned as an array.

In [ ]:
num_samples = 1000
input_size = 784
num_classes = 10

X = np.random.randn(num_samples, input_size)
y = np.random.randint(0, num_classes, size=(num_samples,))

In [ ]:
X.shape

In [ ]:
hidden_size = 128
num_epochs = 10
learning_rate = 0.1

model = TinyImageClassifier(input_size, hidden_size, num_classes)
model.train(X, y, num_epochs, learning_rate)

# Prepare test data
X_test = np.random.randn(10, input_size)

# Make predictions on test data
predicted_classes = model.predict(X_test)
print("Predicted classes:", predicted_classes)

Training a model can be a computationally intensive and time consuming process, so we always want to save the model 
as we train it by creating checkpoints, or as soon as we finish training it by saving the whole model. Since this is 
a toy example, we will go for the latter using the `pickle` module.

In [ ]:
import pickle

In [ ]:
with open("numpy_model.pkl", "wb") as file:
    pickle.dump(model, file)

In [ ]:
with open("numpy_model.pkl", "rb") as file:
    numpy_model = pickle.load(file)

In [ ]:
numpy_model.predict(X_test)

Note: Saving models as pickle files is not adviceable anymore and there are already better ways of doing this using 
[joblib](https://joblib.readthedocs.io/), [safetensor](https://huggingface.co/docs/safetensors/en/index), [ONNX](https://onnx.ai), 
and other tools.

### 3.2 PyTorch Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class TinyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TinyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# Generate synthetic data
num_samples = 1000
input_size = 784
num_classes = 10
hidden_size = 128

In [ ]:
X = torch.randn(num_samples, input_size)
y = torch.randint(0, num_classes, size=(num_samples,))

In [ ]:
model = TinyModel(input_size, hidden_size, num_classes)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Save model

In [ ]:
torch.save(model, 'model_weights.pth')

In [ ]:
model = torch.load('model_weights.pth')
model.eval()

Explanation of the PyTorch code:

We generate synthetic data (X and y) similar to the NumPy example.
We define the model architecture using the nn.Module class, specifying the layers and activation functions.
The forward method defines the forward pass of the model.
We create an instance of the model, specifying the input size, hidden size, and number of classes.
We define the loss function (cross-entropy loss) and the optimizer (stochastic gradient descent).
In the training loop, we perform forward and backward passes, compute the loss, and update the model parameters using the optimizer.
PyTorch automatically computes the gradients during the backward pass using automatic differentiation.

Using the Tiny Foundation Model with Prompts
Once the tiny foundation model is trained, we can use it for various tasks by providing appropriate prompts. For example, let's say we want to classify an image of a handwritten digit. We can create a prompt that guides the model to focus on the relevant features and make a prediction.
Prompt: "Classify the handwritten digit in the image. Look for the overall shape, stroke thickness, and any distinguishing characteristics."
By providing such prompts, we can guide the model to perform specific tasks, even if it was not explicitly trained for them. This is the power of foundation models – their ability to adapt to new tasks with minimal fine-tuning.

## 4. Transfer Learning

![mad_scientist](https://cdn.midjourney.com/b828cd26-b6a7-4086-bb1e-b3c96cf04b8e/0_2.png)

Transfer learning is a machine learning technique that leverages knowledge gained from solving one problem and applies it to a different but related problem. The key idea behind transfer learning is to use pre-trained models, which have been trained on large datasets for a specific task, as a starting point for a new task with limited data or resources.
In traditional machine learning, models are trained from scratch on a specific dataset for a particular task. This requires a large amount of labeled data and computational resources. Transfer learning, on the other hand, allows us to take advantage of the learned features and patterns from a pre-trained model and adapt them to a new task, even if the new task has a different objective or domain.
The process of transfer learning typically involves the following steps:

1. Select a pre-trained model: Choose a model that has been trained on a large dataset for a task similar to the target task. Popular pre-trained models include ResNet, VGG, and BERT, which have been trained on datasets like ImageNet or large text corpora.
2. Freeze or fine-tune layers: Depending on the similarity between the source and target tasks, you may choose to freeze some or all of the layers in the pre-trained model. Freezing layers means keeping their weights fixed during training, while fine-tuning allows the weights to be updated for the new task.
3. Modify the output layer: Replace the output layer of the pre-trained model with a new layer suitable for the target task. For example, if the pre-trained model was used for image classification with 1000 classes and the new task has 10 classes, you would replace the final layer with a new layer having 10 output units.
4. Train the model: Train the modified model on the target task dataset. Since the pre-trained model already has learned features, the training process is typically faster and requires less data compared to training from scratch.
5. Evaluate and iterate: Assess the performance of the model on the target task and iterate by adjusting hyperparameters, modifying the architecture, or trying different pre-trained models until satisfactory results are achieved.

Transfer learning has been successfully applied in various domains, including computer vision, natural language processing, and speech recognition. It has enabled the development of high-performing models even with limited labeled data, making it a valuable technique in scenarios where data acquisition is costly or time-consuming.
Metaphor for the Layman:
Imagine you are a chef who specializes in Italian cuisine. You have spent years perfecting your pasta-making skills and have a deep understanding of Italian flavors and techniques. Now, you want to expand your repertoire and learn to cook Mexican dishes.
Instead of starting from scratch and learning everything about Mexican cuisine from the beginning, you can apply your existing knowledge and skills to adapt to the new cuisine. You already know how to cook pasta, so you can use that knowledge to make similar dishes like tacos or burritos using tortillas instead of pasta. You understand the importance of balancing flavors, so you can apply that principle to create delicious Mexican sauces and seasonings.

### NumPy Example

In [ ]:
import numpy as np

class TinyImageClassifier:
    def __init__(self, input_size, hidden_size, num_classes, pretrained_weights=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        if pretrained_weights is None:
            self.W1 = np.random.randn(input_size, hidden_size) * 0.01
            self.b1 = np.zeros((1, hidden_size))
        else:
            self.W1 = pretrained_weights['W1']
            self.b1 = pretrained_weights['b1']
        
        self.W2 = np.random.randn(hidden_size, num_classes) * 0.01
        self.b2 = np.zeros((1, num_classes))
    
    def forward(self, X):
        self.h = np.maximum(0, np.dot(X, self.W1) + self.b1)
        scores = np.dot(self.h, self.W2) + self.b2
        return scores
    
    def backward(self, X, y, scores):
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        num_samples = X.shape[0]
        
        dscores = probs
        dscores[range(num_samples), y] -= 1
        dscores /= num_samples
        
        dW2 = np.dot(self.h.T, dscores)
        db2 = np.sum(dscores, axis=0, keepdims=True)
        
        dh = np.dot(dscores, self.W2.T)
        dh[self.h <= 0] = 0
        
        dW1 = np.dot(X.T, dh)
        db1 = np.sum(dh, axis=0, keepdims=True)
        
        return dW1, db1, dW2, db2
    
    def train(self, X, y, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            scores = self.forward(X)
            
            exp_scores = np.exp(scores)
            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
            correct_logprobs = -np.log(probs[range(num_samples), y])
            loss = np.sum(correct_logprobs) / num_samples
            
            dW1, db1, dW2, db2 = self.backward(X, y, scores)
            
            self.W1 -= learning_rate * dW1
            self.b1 -= learning_rate * db1
            self.W2 -= learning_rate * dW2
            self.b2 -= learning_rate * db2
            
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")
    
    def predict(self, X):
        scores = self.forward(X)
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return np.argmax(probs, axis=1)

1. We modify the __init__ method of the TinyImageClassifier class to accept an optional pretrained_weights parameter. If pretrained_weights is provided, we initialize the weights (W1 and b1) of the first layer with the pre-trained weights instead of random initialization.
2. We pre-train the model on the original task using the TinyImageClassifier class, as we did before. This step trains the model on the original dataset (X_train and y_train) and learns the weights (W1 and b1) for the first layer.
3. F or transfer learning, we create a new instance of the TinyImageClassifier class called new_model. We specify the input size, a new hidden size (new_hidden_size), and the number of classes for the new task (new_num_classes).
4. We pass the pre-trained weights (pretrained_weights) from the first layer of the pre-trained model to the new_model. This initializes the weights of the first layer in the new_model with the learned weights from the pre-trained model.
5. We fine-tune the new_model on the new task using a new dataset (X_new and y_new). We typically use a smaller learning rate and fewer epochs for fine-tuning compared to the original training.
6. After fine-tuning, we can use the predict method of the new_model to make predictions on new samples (X_test) for the new task.

By using the pre-trained weights from the first layer of the original model and fine-tuning them for the new task, we leverage the learned features and adapt them to the specific requirements of the new classification problem. This allows us to benefit from the knowledge learned in the original task and potentially achieve better performance on the new task with less training data and fewer iterations.

In [ ]:
# Pre-train the model on the original task
num_samples = 1000
input_size = 784
hidden_size = 128
num_classes = 10

X_train = np.random.randn(num_samples, input_size)
y_train = np.random.randint(0, num_classes, size=(num_samples,))

pretrained_model = TinyImageClassifier(input_size, hidden_size, num_classes)
pretrained_model.train(X_train, y_train, num_epochs=10, learning_rate=0.1)

# Transfer learning for a new task
new_num_classes = 5
new_hidden_size = 64

pretrained_weights = {
    'W1': pretrained_model.W1,
    'b1': pretrained_model.b1
}

new_model = TinyImageClassifier(input_size, new_hidden_size, new_num_classes, pretrained_weights)

# Fine-tune the model on the new task
X_new = np.random.randn(num_samples, input_size)
y_new = np.random.randint(0, new_num_classes, size=(num_samples,))

new_model.train(X_new, y_new, num_epochs=5, learning_rate=0.01)

# Make predictions on new samples
X_test = np.random.randn(10, input_size)
predicted_classes = new_model.predict(X_test)
print("Predicted classes:", predicted_classes)

### PyTorch Example

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        out = self.fc(hidden.squeeze(0))
        return out

class SummarizationModel(nn.Module):
    def __init__(self, pretrained_model, output_size):
        super(SummarizationModel, self).__init__()
        self.embedding = pretrained_model.embedding
        self.lstm = pretrained_model.lstm
        self.fc = nn.Linear(pretrained_model.lstm.hidden_size, output_size)
        
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        out = self.fc(hidden.squeeze(0))
        return out

1. We define a new class called SummarizationModel that inherits from nn.Module. This model will be used for the summarization task.
2. In the __init__ method of SummarizationModel, we take the pre-trained sentiment analysis model (pretrained_model) as a parameter. We initialize the embedding layer and LSTM layer of the summarization model with the corresponding layers from the pre-trained model. This allows us to transfer the learned weights from the sentiment analysis task to the summarization task.
3. We replace the final fully connected layer (fc) of the summarization model with a new layer that outputs the desired vocabulary size for summarization (output_size).
4. The forward method of SummarizationModel remains similar to the sentiment analysis model, except for the output size.
5. We create an instance of the pre-trained sentiment analysis model (sentiment_model) using the same hyperparameters as before.
6. We create an instance of the summarization model (summarization_model) by passing the pre-trained sentiment model and the desired output size.
7. We fine-tune the summarization model using a new optimizer and loss function specific to the summarization task. We assume that you have the input sequences (X) and corresponding target summaries (y) for training.
8. The training loop is similar to the sentiment analysis task, but now we use the summarization_model and the new optimizer and loss function.
9. After training, we can use the fine-tuned summarization_model to make predictions on new input sequences (X_test) for summarization. The predicted summaries are obtained by taking the argmax of the model's outputs.

By leveraging the pre-trained embeddings and LSTM layers from the sentiment analysis model, we can transfer the learned knowledge to the summarization task. This allows the model to capture important features and patterns from the sentiment analysis task that can be beneficial for summarization.

Note that this is a simplified example, and in practice, you may need to make additional modifications based on the specific requirements of your summarization task, such as handling variable-length sequences, using attention mechanisms, or employing more advanced architectures like transformer models.

In [ ]:
# Pre-train the sentiment analysis model
vocab_size = 5000
embedding_dim = 128
hidden_size = 256
num_classes = 2

sentiment_model = SentimentModel(vocab_size, embedding_dim, hidden_size, num_classes)

# Transfer learning for summarization
output_size = 1000  # Vocabulary size for summarization
summarization_model = SummarizationModel(sentiment_model, output_size)

# Fine-tune the summarization model
optimizer = optim.Adam(summarization_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X), batch_size):
        batch_X = X[i:i+batch_size]
        batch_y = y[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = summarization_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Make predictions on new samples
X_test = ...  # New input sequences for summarization
outputs = summarization_model(X_test)
predicted_summaries = outputs.argmax(dim=1)

## 5. Transformers

![attention_meme](https://i.kym-cdn.com/entries/icons/original/000/036/585/Attention_is_all_you_need.jpg)  
Source: [Know Your Meme](https://knowyourmeme.com/memes/attention-is-all-you-need)


The Transformers architecture, introduced in the paper ["Attention Is All You Need" by Vaswani et al.](https://arxiv.org/abs/1706.03762), 
has revolutionized the field of natural language processing (NLP) and has since been applied to various other 
domains, including computer vision and audio. The key innovation of the Transformers architecture is the 
self-attention mechanism, which allows the model to weigh the importance of different parts of the input 
sequence when making predictions.

The Transformers architecture consists of an encoder and a decoder, each composed of multiple layers. The 
encoder takes the input sequence and generates a contextualized representation, while the decoder generates the 
output sequence based on the encoder's output and the previous outputs.

The main components of the Transformers architecture are:

1. Embedding Layer: The input tokens are converted into dense vector representations using an embedding 
layer. Positional encodings are added to the embeddings to capture the sequential nature of the input.
2. Multi-Head Attention: The self-attention mechanism is applied through multi-head attention. The input 
sequence is linearly projected into query, key, and value vectors. The attention scores are computed by 
taking the dot product of the query and key vectors, which determines the importance of each token in the
sequence. The attention scores are then used to weight the value vectors, resulting in a weighted sum that
captures the relevant information.
3. Feed-Forward Neural Network: Each layer in the encoder and decoder also includes a position-wise feed-forward 
neural network. This network consists of two linear transformations with a ReLU activation in between, applied
independently to each position in the sequence.
6. Layer Normalization and Residual Connections: Layer normalization is applied after each sub-layer (multi-head 
attention and feed-forward neural network) to normalize the activations and stabilize training. Residual connections
are used to facilitate the flow of information and gradients through the network.
8. Decoder: The decoder follows a similar structure to the encoder but includes an additional multi-head attention 
layer that attends to the encoder's output. This allows the decoder to focus on relevant parts of the input sequence
when generating the output.

The Transformers architecture has several advantages over previous approaches like recurrent neural networks (RNNs) 
and convolutional neural networks (CNNs). It can handle long-range dependencies effectively, allows for parallel 
computation, and scales well to large datasets. The self-attention mechanism enables the model to capture complex 
relationships between tokens in the sequence, leading to improved performance on various NLP tasks such as machine 
translation, text summarization, and sentiment analysis.

A better, and more fun way to think about them is via the following analogy.

Imagine you are a detective trying to solve a complex case. You have a large pile of documents containing 
information about the case, and your task is to find the relevant pieces of information to crack the case.

Instead of reading the documents sequentially from beginning to end, you decide to use a smart approach. You 
create multiple copies of yourself (multi-head attention) and assign each copy to focus on different aspects 
of the documents. One copy looks for names, another looks for dates, and another looks for locations. Each 
copy weighs the importance of each piece of information based on its relevance to the case.

After gathering the important information, you and your copies discuss and combine your findings (feed-forward 
neural network). You then organize and summarize the key points (layer normalization) and add them to your 
existing knowledge about the case (residual connections).

You repeat this process multiple times, each time refining your understanding of the case by focusing on different 
aspects and combining the information in a meaningful way. Finally, you use all the gathered knowledge to generate 
a coherent report that solves the case.

### 5.1 Tokenization

Tokenization is a crucial step in preparing the input data for a Transformer model. It is the process of breaking 
down a piece of text into smaller units called tokens, which can be individual words, subwords, or even characters. The 
purpose of tokenization is to convert the raw text into a format that can be easily processed and understood by the model.

The tokenization process typically involves the following steps:

1. Text Cleaning: The raw text is cleaned by removing unwanted characters, such as punctuation marks, special characters, or HTML tags, depending on the requirements of the task.
2. Text Splitting: The cleaned text is split into smaller units called tokens. This can be done using various techniques, such as:
   - Word-based tokenization: The text is split into individual words based on whitespace or punctuation.
   - Subword tokenization: The text is split into subword units, which can be smaller than words. This helps in handling out-of-vocabulary (OOV) words and reduces the size of the vocabulary. Common subword tokenization algorithms include Byte-Pair Encoding (BPE) and WordPiece.
   - Character-based tokenization: The text is split into individual characters, which can be useful for tasks like character-level language modeling.
3. Vocabulary Creation: A vocabulary is created based on the tokens obtained from the training data. The vocabulary assigns a unique integer ID to each token, allowing the model to work with numerical representations.
4. Token Encoding: Each token in the input text is replaced with its corresponding integer ID from the vocabulary. This step converts the text into a sequence of integers that can be fed into the Transformer model.
5. Special Tokens: Additional special tokens are added to the input sequence to provide extra information to the model. Common special tokens include:
   - `[CLS]`: A special token added at the beginning of the sequence, typically used for classification tasks.
   - `[SEP]`: A special token used to separate different parts of the input, such as sentences or documents.
   - `[PAD]`: A special token used for padding the input sequence to a fixed length.
   - `[MASK]`: A special token used for masked language modeling tasks, indicating the position of masked tokens.
6. Input Formatting: The tokenized and encoded input is formatted into the required input format for the Transformer model, such as a tensor of shape (batch_size, sequence_length).


Another way to think of tokenization.

Tokenization is like breaking down a large puzzle into smaller, manageable pieces. Imagine you have a 1000-piece puzzle (the raw text), and your goal is to assemble it (process it with the Transformer model). However, working with the entire puzzle at once can be overwhelming and inefficient.

So, you start by sorting the puzzle pieces (text cleaning) and grouping them based on their characteristics, such as color or shape (text splitting). You then assign a unique label to each group of pieces (vocabulary creation) to keep track of them easily.

Next, you replace each puzzle piece with its corresponding label (token encoding) and add special markers (special tokens) to indicate the beginning, end, or separation of different sections of the puzzle.

Finally, you arrange the labeled pieces in a structured manner (input formatting) so that you can easily work with them and start assembling the puzzle (feeding the input to the Transformer model).

By breaking down the large puzzle into smaller, labeled pieces, you make the task more manageable and efficient, just like how tokenization helps in processing and understanding text data in a Transformer model.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

In [ ]:
text = "Today we are learning about foundation models!"
tokens = tokenizer.tokenize(text)
print(tokens)

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

In [ ]:
decoded_text = tokenizer.decode(token_ids)
print(decoded_text)

### 5.2 Training Process

Training a Transformer model involves the following steps:
1. Prepare the input data by tokenizing and converting tokens to dense vector representations using an embedding layer.
2. Add positional encodings to capture the sequential nature of the input.
3. Pass the input through the encoder, which applies multi-head attention and feed-forward neural networks to generate contextualized representations.
4. Pass the encoder's output and previous decoder outputs through the decoder, which generates the output sequence using multi-head attention and feed-forward neural networks.
5. Apply layer normalization and residual connections to stabilize training and facilitate information flow.
6. Compute the loss function, typically cross-entropy loss, to measure the difference between predicted and target outputs.
7. Use an optimizer, such as Adam, to update the model's parameters based on the gradients of the loss function.
8. Repeat steps 3-7 for multiple epochs until the model converges or reaches the desired performance.

Since training a transformer model is more involved in terms of code complexity and time taken to finish, we will 
evaluate several training implementations of a transformer.
- [hf Q&A example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py)

### 5.3 Prompting

Prompting is a technique used to guide the Transformer model to perform specific tasks without fine-tuning. It involves providing a task-specific prompt or instruction along with the input sequence. The model then generates the output based on the given prompt. Prompting is effective for tasks such as text generation, question answering, and sentiment analysis. It allows for quick adaptation to new tasks without the need for extensive training data or model modifications.

We can create prompts with straightforward text.

We can also use specialized tools like guidance, outlines, sglang, dspy or instructor.

We can enhance prompts using Retrieval Augmented Generation.

The most important thing to keep in mind is the prompt structure your model is expecting (if it is a text-to-something model).

Some examples include:

### 5.4 Fine-Tuning

Fine-tuning is the process of adapting a pre-trained Transformer model to a specific downstream task. It involves the following steps:
1. Initialize the model with pre-trained weights from a large-scale corpus.
2. Replace the original output layer with a new layer specific to the downstream task.
3. Freeze the weights of the pre-trained layers to preserve the learned representations.
4. Train the model on the downstream task dataset, updating only the weights of the new output layer and optionally the top few layers of the model.
5. Evaluate the fine-tuned model on the task-specific validation set and adjust hyperparameters if necessary.
Fine-tuning allows the model to leverage the knowledge learned from pre-training and adapt it to specific tasks with limited training data.

Example using https://www.kaggle.com/datasets/jorgeruizdev/ludwig-music-dataset-moods-and-subgenres

In [ ]:
from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import pipeline

In [ ]:
dataset = load_dataset(
    "audiofolder", 
    data_dir="data/",
    drop_metadata=True,
    split="train"
)

In [ ]:
dataset

In [ ]:
dataset.features["label"]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

In [ ]:
dataset["train"].features

In [ ]:
dataset["train"].features["label"]

In [ ]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

label2id, id2label

In [ ]:
id2label[str(2)]

To fine tune a music classifier, we will use [wav2vec](https://arxiv.org/abs/2006.11477) from Meta as a our base model.

> Wav2vec is a self-supervised speech recognition model developed by Meta AI (formerly Facebook AI) that can learn powerful speech representations from raw audio data with little to no transcribed speech.

The key aspects of wav2vec are:
- Wav2vec uses a self-supervised approach to learn speech representations directly from raw audio without relying on large amounts of transcribed speech data.
- Similar to BERT for text, wav2vec is trained by masking parts of the raw audio and learning to predict the masked speech units.
- Wav2vec automatically discovers speech units (shorter than phonemes) that are used to represent the speech audio sequence.
- By leveraging large amounts of unlabeled speech data through self-supervision, wav2vec can outperform traditional supervised speech recognition models that rely solely on transcribed audio, even when using 100 times less labeled data.

The key motivation behind wav2vec is to make speech technology accessible to more languages and dialects by reducing the dependence on transcribed data, which is scarce for most languages. Meta AI has open-sourced the pretrained models and code to enable wider adoption and research in this area.

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

In [ ]:
feature_extractor.sampling_rate

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
dataset["train"][0]

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    return feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True, padding=True
    )

In [ ]:
encoded_latin = dataset.map(preprocess_function, remove_columns="audio", batched=True)

In [ ]:
encoded_latin

In [ ]:
encoded_latin["train"].features["input_values"]

In [ ]:
print(encoded_latin["train"][:1])

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
num_labels = len(id2label)
num_labels, label2id, id2label

In [ ]:
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

In [ ]:
training_args = TrainingArguments(
    output_dir="models",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_latin["train"],
    eval_dataset=encoded_latin["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
%%time

trainer.train()

In [ ]:
trainer.save_model("models")

In [ ]:
classifier = pipeline("audio-classification", model="models")

In [ ]:
from random import choice
audio_file = dataset["train"][choice(range(5))]["audio"]["path"]
audio_file

In [ ]:
classifier.predict(audio_file)

In [ ]:
from IPython.display import Audio as Audio2

Audio2(audio_file)

It works, not well, but it works! 😎👌🔥

There are many other thing to keep in mind or to do while fine-tuning, but going through these is out of the scope of this session so I highly encourage you to check out the resources below.

### 5.5 Reinforcement Learning with Human Feedback (RLHF)

![rlhf](https://assets-global.website-files.com/63024b20439fa6bd66ee3465/657a79cf031c15004cc74699_Thumbnail.PNG)

RLHF is an approach to align language models with human preferences. It involves the following steps:
1. Collect a dataset of human-generated responses to a given set of prompts.
2. Train a reward model to predict the quality of the model's outputs based on human feedback.
3. Use the reward model to provide feedback to the language model during the training process.
4. Update the language model's parameters based on the rewards received, encouraging it to generate outputs that align with human preferences.
RLHF helps to mitigate issues like bias, toxicity, and hallucinations in generated outputs, making the models more reliable and safe for real-world applications.

RLFH can be a much more involved a step as the triaining of the model itself, therefore, we won't be cover this step here but 
you will see resources below for you to learn more about RLHF.
- [LLM Training: RLHF and Its Alternatives by SEBASTIAN RASCHKA, PHD](https://magazine.sebastianraschka.com/p/llm-training-rlhf-and-its-alternatives)
- [Stanford CS224N | 2023 | Lecture 10 - Prompting, Reinforcement Learning from Human Feedback](https://www.youtube.com/watch?v=SXpJ9EmG3s4)
- [Reinforcement Learning from Human Feedback: From Zero to chatGPT](https://www.youtube.com/watch?v=2MBJOuVq380)
- [Reinforcement Learning from Human Feedback course by DeepLearning.ai](https://www.deeplearning.ai/short-courses/reinforcement-learning-from-human-feedback)

### 5.6 Challenges

Despite the success of Transformers, there are several challenges:

1. Limited Context: Transformers have a fixed context window, limiting their ability to process long-range dependencies beyond the window size.

![context window](https://i0.wp.com/bdtechtalks.com/wp-content/uploads/2023/11/robot-llm-long-context.jpg?ssl=1)  
Source: [StreamingLLM gives language models unlimited context by Ben Dickson](https://bdtechtalks.com/2023/11/27/streamingllm/)

2. Lack of Interpretability: The complex nature of self-attention makes it difficult to interpret the model's decisions and reasoning.

![explainthyself](https://miro.medium.com/v2/resize:fit:2000/1*nJinotXOj6DY4xtss2cxpA.png)

3. Bias and Fairness: Transformer models can inherit biases present in the training data, leading to biased or unfair outputs.

![bias](https://d2eehagpk5cl65.cloudfront.net/img/c800x450-w800-q80/uploads/2024/02/Screenshot-2024-02-22-at-8.18.28-AM-800x450.png)

4. Out-of-Distribution Generalization (aka Hallucinations): Transformers may struggle to generalize well to data that is significantly different from the training distribution.

5. Out-of-Date Knowledge

![out_of_date](https://global.discourse-cdn.com/openai1/original/3X/5/6/56b607844d56bd6017ce7a9d8b39fb557becdb9b.png)

6. Batching

![challenge](https://images.ctfassets.net/xjan103pcp94/1LJioEsEdQQpDCxYNWirU6/82b9fbfc5b78b10c1d4508b60e72fdcf/cb_02_diagram-static-batching.png)

7. Infrastructure: Training Transformer models requires significant computational resources due to their large size and the need for extensive pre-training.
![costs](https://www.nextplatform.com/wp-content/uploads/2022/12/cerebras-model-training-price-table-TNP.jpg)  
Source: [Counting the Cost of Training Large Language Models by Timothy Prickett Morgan](https://www.nextplatform.com/2022/12/01/counting-the-cost-of-training-large-language-models/)
    1. Costs
    2. Scarcity
    3. Expertise
    4. Lock-In

Researchers are actively working on addressing these challenges to improve the efficiency, 
interpretability, and robustness of these models.

## 7. Serving

Strategies for Deployment and Post-Deployment Maintenance



Here's the process expressed as a mermaid diagram:

```mermaid
graph LR
    A[Collect Data] --> B[Engineer Features]
    B --> C[Train and Evaluate the Model]
    C --> D[Evaluate Model]
    D --> B
    D --> E[Serialize and Package the Model]
    D --> F[Choose a Deployment Architecture]
    E --> G[Containerize the Model]
    F --> G
    G --> H[Deploy the Model]
    H --> I[Expose the Model Endpoint]
    I --> J[Monitor and Maintain]
    J --> B
```

This diagram illustrates the high-level steps involved in the machine learning 
lifecycle, from training and evaluation to deployment, exposure, and maintenance. Each 
step plays a crucial role in ensuring the model is effectively served and can be 
accessed by the intended consumers.

In [ ]:
from transformers import pipeline

In [ ]:
model = pipeline("summarization", model="t5-small")

In [ ]:
model(["this is a long story from a king 1000 years ago"])

In [ ]:
!mkdir -p servers/summarizer

![mlserver](https://mlserver.readthedocs.io/en/latest/_images/mlserver_setup.png)

In [ ]:
%%writefile servers/summarizer/t5_model.py

from mlserver import MLModel
from mlserver.codecs import decode_args
from transformers import pipeline
from typing import List

class Summarizer(MLModel):
    async def load(self):
        self.model = pipeline("summarization", model="t5-small", device=)

    @decode_args
    async def predict(self, text: List[str]) -> List[str]:
        return [model(text)['summary_text']]

In [ ]:
%%writefile servers/summarizer/model-settings.json
{
    "name": "summarizer",
    "implementation": "t5_model.Summarizer"
}

From the terminal, start your service with the following command.

```sh
mlserver start servers/summarizer
```

Now we can test our service.

In [ ]:
import wikipediaapi

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('MyMovieEval (example@example.com)', 'en')

In [ ]:
barbie = wiki_wiki.page('Barbie_(film)').summary
oppenheimer = wiki_wiki.page('Oppenheimer_(film)').summary

print(barbie)
print()
print(oppenheimer)

In [ ]:
import requests
inference_request = {
    "inputs": [
        {
          "name": "text_inputs",
          "shape": [1],
          "datatype": "BYTES",
          "data": [article],
        }
    ]
}

In [ ]:
endpoint = "http://localhost:8080/v2/models/summarizer/infer"

In [ ]:
r = requests.post(endpoint, json=inference_request)

In [ ]:
r.json()

In [ ]:
%%writefile servers/translator/en_to_es.py

from mlserver import MLModel
from mlserver.codecs import decode_args
from transformers import pipeline
from typing import List

class Translator(MLModel):
    async def load(self):
        self.model = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

    @decode_args
    async def predict(self, payload: List[str]) -> List[str]:
        model_output = self.model(payload, min_length=5, max_length=100)
        return [model_output[0]['translation_text']]

In [ ]:
%%writefile servers/translator/model-settings.json
{
    "name": "translator",
    "implementation": "en_to_es.Translator"
}

In [ ]:
%%writefile models/settings.json
{
    "http_port": 7070,
    "grpc_port": 7090,
    "metrics_port": 7080
}

From the terminal, start your service with the following command.

```sh
mlserver start servers
```

Now we can test our service.

In [ ]:
endpoint = "http://localhost:8080/v2/models/summarizer/infer"

In [ ]:
r = requests.post(endpoint, json=inference_request)

In [ ]:
r.json()

## 8. What we are up to at Seldon

To tackle some of the challenges we touched on (i.e., )

![slide3](images/slide_3.png)
![slide4](images/slide_4.png)
![slide5](images/slide_5.png)
![slide6](images/slide_6.png)
![slide7](images/slide_7.png)
![slide8](images/slide_8.png)
![slide9](images/slide_9.png)
![slide10](images/slide_10.png)
![slide11](images/slide_11.png)
![slide12](images/slide_12.png)
![slide13](images/slide_13.png)
![slide14](images/slide_14.png)

## 9. Conclusion






If anymore questions come up, of if you'd just like to chat about MLOps, please join our community 
[at the link here](https://seldondev.slack.com/join/shared_invite/zt-vejg6ttd-ksZiQs3O_HOtPQsen_labg#/shared-invite/email).